In [2]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv(r"data\unlabelled\santacruz_cleaned.csv")
df

In [ ]:
normals = {
    1: 30.4, 2: 31.1, 3: 32.6, 4: 33.0, 5: 33.3, 6: 32.0,
    7: 29.9, 8: 29.5, 9: 30.2, 10: 33.1, 11: 33.4, 12: 31.9
}
df.insert(loc=7, column='Normal_Temp', value=df['MN'].map(normals))

In [ ]:
df

In [ ]:
df.to_csv(r"data\unlabelled\santacuz_with_normal.csv")

In [ ]:
daily_max = df.groupby(['YEAR', 'MN', 'DT'])['DBT'].max().reset_index()
thresholds = {month: normal + 4.5 for month, normal in normals.items()}

# Function to determine heatwave label
def is_heatwave(dbt, month):
    if dbt >= 37.0 or dbt >= thresholds[month]:
        return 1
    return 0

# Merge daily max DBT back to original dataframe
df = df.merge(daily_max[['YEAR', 'MN', 'DT', 'DBT']], on=['YEAR', 'MN', 'DT'], suffixes=('', '_max'))
df['Heatwave'] = df.apply(lambda row: is_heatwave(row['DBT_max'], row['MN']), axis=1)
df = df.drop(columns=['DBT_max'])  # Remove temporary max column
print(thresholds)

In [ ]:
df

In [ ]:
heatwave_counts = df['Heatwave'].value_counts()
heatwave_counts

In [ ]:
if os.path.exists(r"data\labelled\santacruz_labelled.csv"):
    print("Labelled processed data exists")
else:
    df.to_csv(r"data\labelled\santacruz_labelled.csv", index= False)
    print("Labelled processed data created")

In [ ]:
df['DATE'] = pd.to_datetime(df[['YEAR', 'MN', 'DT']].rename(columns={'MN': 'month', 'DT': 'day'}))
df.set_index('DATE', inplace=True)
df

In [ ]:
df.drop(columns=['YEAR', 'MN', 'DT', 'HR'], inplace=True)


In [ ]:
daily_df = df.resample('D').mean()


In [ ]:
if os.path.exists(r"data\labelled\santacruz_labelled_daily.csv"):
    print("Labelled processed data exists")
else:
    daily_df.to_csv(r"data\labelled\santacruz_labelled_daily.csv")
    print("Labelled processed data created")

In [13]:
df_daily = pd.read_csv(r"data\labelled\santacruz_labelled_daily.csv")

In [14]:
df_daily

,DATE,DPT,WBT,DBT,Normal_Temp,Heatwave
0,2010-01-01,16.0875,19.775,25.725000,30.4,0.0
1,2010-01-02,18.2000,21.375,26.975000,30.4,0.0
2,2010-01-03,19.7375,22.025,26.300000,30.4,0.0
3,2010-01-04,18.0250,20.450,24.575000,30.4,0.0
4,2010-01-05,18.5500,20.575,24.175000,30.4,0.0
...,...,...,...,...,...,...
5474,2024-12-27,NaN,NaN,21.847186,31.9,0.0
5475,2024-12-28,NaN,NaN,21.261927,31.9,0.0
5476,2024-12-29,NaN,NaN,21.481941,31.9,0.0
5477,2024-12-30,NaN,NaN,21.161551,31.9,0.0


In [15]:
df_daily = df_daily.interpolate()


C:\Users\LaukikS\AppData\Local\Temp\ipykernel_15824\3331088670.py:1: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df_daily = df_daily.interpolate()


In [16]:
df_daily.isnull().sum()

DATE           0
DPT            0
WBT            0
DBT            0
Normal_Temp    0
Heatwave       0
dtype: int64

In [17]:
heatwave_counts_final = df_daily['Heatwave'].value_counts()
heatwave_counts_final

Heatwave
0.0    5379
1.0     100
Name: count, dtype: int64

In [18]:
new_df = pd.read_csv(r"data\labelled\santacruz_labelled_daily.csv")
new_df

,DATE,DPT,WBT,DBT,Normal_Temp,Heatwave
0,2010-01-01,16.0875,19.775,25.725000,30.4,0.0
1,2010-01-02,18.2000,21.375,26.975000,30.4,0.0
2,2010-01-03,19.7375,22.025,26.300000,30.4,0.0
3,2010-01-04,18.0250,20.450,24.575000,30.4,0.0
4,2010-01-05,18.5500,20.575,24.175000,30.4,0.0
...,...,...,...,...,...,...
5474,2024-12-27,NaN,NaN,21.847186,31.9,0.0
5475,2024-12-28,NaN,NaN,21.261927,31.9,0.0
5476,2024-12-29,NaN,NaN,21.481941,31.9,0.0
5477,2024-12-30,NaN,NaN,21.161551,31.9,0.0


In [19]:

new_df['heatwave_days'] = 0
consecutive_days = (new_df['Heatwave'] == 1) & (new_df['Heatwave'].shift(1) == 1)

# Assign 1 to the identified consecutive heatwave days
new_df.loc[consecutive_days | consecutive_days.shift(-1, fill_value=False), 'heatwave_days'] = 1

In [20]:
new_df

,DATE,DPT,WBT,DBT,Normal_Temp,Heatwave,heatwave_days
0,2010-01-01,16.0875,19.775,25.725000,30.4,0.0,0
1,2010-01-02,18.2000,21.375,26.975000,30.4,0.0,0
2,2010-01-03,19.7375,22.025,26.300000,30.4,0.0,0
3,2010-01-04,18.0250,20.450,24.575000,30.4,0.0,0
4,2010-01-05,18.5500,20.575,24.175000,30.4,0.0,0
...,...,...,...,...,...,...,...
5474,2024-12-27,NaN,NaN,21.847186,31.9,0.0,0
5475,2024-12-28,NaN,NaN,21.261927,31.9,0.0,0
5476,2024-12-29,NaN,NaN,21.481941,31.9,0.0,0
5477,2024-12-30,NaN,NaN,21.161551,31.9,0.0,0


In [21]:
heatwave_counts_days = new_df['Heatwave'].value_counts()
heatwave_counts_days

Heatwave
0.0    5370
1.0     100
Name: count, dtype: int64

In [22]:
new_df.to_csv(r"data\labelled\santacruz_labelled_days.csv", index = False)

In [23]:
df = pd.read_csv(r'data\labelled\santacruz_labelled_days.csv')

In [24]:
heatwave_counts = df['Heatwave'].value_counts()
heatwave_counts

Heatwave
0.0    5370
1.0     100
Name: count, dtype: int64